In [1]:
import sys, os
import subprocess
import threading

sys.path.insert(0,"../scripts")
import silence_splitter as ss

In [2]:
root_dir = '../tmp/'

In [ ]:
relative_file_paths = []
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".mp3"):
        	relative_file_paths.append(os.path.join(root, file))

In [ ]:
print relative_file_paths

In [ ]:
subs_map = {}
subs_map[' '] = '_'
subs_map['á'] = 'a'
subs_map['é'] = 'e'
subs_map['í'] = 'i'
subs_map['ó'] = 'o'
subs_map['ö'] = 'o'
subs_map['ő'] = 'o'
subs_map['ú'] = 'u'
subs_map['ü'] = 'u'
subs_map['ű'] = 'u'

def generate_new_path(text, old_root, new_root):
    new_text = text.replace(old_root, new_root)
    new_text = str.lower(new_text)
    for val in subs_map:
        new_text = new_text.replace(val,subs_map[val])
    return new_text

In [ ]:
new_relative_file_paths = []
for path in relative_file_paths:
    new_relative_file_paths.append(generate_new_path(path,'tmp','tmp_subs'))

In [ ]:
#print new_relative_file_paths

In [ ]:
for i in xrange(len(relative_file_paths)):
    f_name = new_relative_file_paths[i].split("/")[-1]
    mkdir_command = 'mkdir -p %s' % new_relative_file_paths[i][0:-len(f_name)]
    subprocess.call(mkdir_command.split(' '))
    subprocess.call(['cp',relative_file_paths[i],new_relative_file_paths[i]])

In [ ]:
max_num_parallel = 4

min_silence_length = 500
silence_threshold = 26

threads = []
for i in xrange(len(new_relative_file_paths)):
    threads.append(threading.Thread(target=ss.split_song, args=(new_relative_file_paths[i],min_silence_length,silence_threshold)))
    
#threads = threads[0:8]
all_thread_num = len(threads)
    
thread_index = 0
while thread_index < all_thread_num:
    upper_index = min(thread_index+max_num_parallel, all_thread_num)
    tmp_threads = threads[thread_index:upper_index]
    print thread_index, upper_index
    for th in tmp_threads:
        th.start()
    for th in tmp_threads:
        th.join()
    
    thread_index += max_num_parallel
